In [1]:
import numpy as np
import jax
import jax.numpy as jnp
import scipy.stats as ss

# import matplotlib.pyplot as plt
# %matplotlib inline
# from IPython.display import display
# import sympy; sympy.init_printing()

# def display_matrix(m):
#     display(sympy.Matrix(m))

/Users/carolinepei/Desktop/mfe/project/citi/term_2_citi_project/citi/lib/python3.8/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [2]:
N = 4          # number of time steps
r = 0.06       # interest rate
K = 1.1        # strike 
T = 3          # Maturity

dt = T/(N-1)          # time interval
df = jnp.exp(-r * dt)  # discount factor per time interval

In [3]:
S = jnp.array([
            [1.00, 1.09, 1.08, 1.34],
            [1.00, 1.16, 1.26, 1.54],
            [1.00, 1.22, 1.07, 1.03],
            [1.00, 0.93, 0.97, 0.92],
            [1.00, 1.11, 1.56, 1.52],
            [1.00, 0.76, 0.77, 0.90],
            [1.00, 0.92, 0.84, 1.01],
            [1.00, 0.88, 1.22, 1.34]])

In [5]:
H = jnp.maximum(K - S, 0)
# Htmp = np.maximum(K - S, 0)           # intrinsic values for put option

V = np.zeros_like(H)               # value matrix
V[:,-1] = H[:,-1]

# Valuation by LS Method
for t in range(N-2, 0, -1):

    good_paths = H[:,t] > 0        # paths where the intrinsic value is positive 
                                   # the regression is performed only on these paths 
    
    rg = np.polyfit( S[good_paths, t], V[good_paths, t+1] * df, 2)    # polynomial regression
    C = np.polyval( rg, S[good_paths,t] )                             # evaluation of regression  
    
    exercise = np.zeros( len(good_paths), dtype=bool)    # initialize
    print(H[good_paths,t] > C)
#     print(Htmp[good_paths,t] > C)
    print(exercise[good_paths])
    exercise[good_paths] = H[good_paths,t] > C           # paths where it is optimal to exercise
    
    V[exercise,t] = H[exercise,t]                        # set V equal to H where it is optimal to exercise 
    V[exercise,t+1:] = 0                                 # set future cash flows, for that path, equal to zero  
    discount_path = (V[:,t] == 0)                        # paths where we didn't exercise 
    V[discount_path,t] = V[discount_path,t+1] * df       # set V[t] in continuation region
    
V0 = jnp.mean(V[:,1]) * df  # discounted expectation of V[t=1]
print("Example price= ", V0)

[False False  True  True  True]
[]


/var/folders/vj/0h54h9js2ml58mblr67cdwn40000gn/T/ipykernel_32411/3256644564.py:19: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  print(exercise[good_paths])
/var/folders/vj/0h54h9js2ml58mblr67cdwn40000gn/T/ipykernel_32411/3256644564.py:20: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  exercise[good_paths] = H[good_paths,t] > C           # paths where it is optimal to exercise


ValueError: shape mismatch: value array of shape (5,) could not be broadcast to indexing result of shape (0,8)

In [6]:
print(H.round(2))


[[0.09999999 0.01       0.02       0.        ]
 [0.09999999 0.         0.         0.        ]
 [0.09999999 0.         0.03       0.07      ]
 [0.09999999 0.17       0.13       0.17999999]
 [0.09999999 0.         0.         0.        ]
 [0.09999999 0.34       0.32999998 0.19999999]
 [0.09999999 0.17999999 0.26       0.09      ]
 [0.09999999 0.22       0.         0.        ]]
